In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import shutil

In [ ]:
speech_content = pd.read_csv("")
max_trials = 

pathTextGrid = 
noWords, wordTimings = getWordTimings(pathTextGrid, offset = 0, wordStartIdx = 0)

In [ ]:
def folderCheckMake(path):
    if os.path.exists(path) == False:
        os.mkdir(path)

In [ ]:
def getTokens(text):
    to_return = []
    
    temp = text.lower().split()
    for word in temp:
        to_return.append(word.rstrip(',!.?'))
    
    return to_return

In [ ]:
def getWordTimings(pathTextGrid, offset = 0, wordStartIdx = 0):
    
    this_data = {}
    wordCTR = 0
    
    fileNameTextGrid = pathTextGrid
    thisTextGrid = textgrids.TextGrid(fileNameTextGrid)
    
    for wordNo, wordData in enumerate(thisTextGrid['words']):
        
        this_word_data = {}
        
        this_word_data['word'] = wordData.text
        this_word_data['start'] = wordData.xmin + offset
        this_word_data['stop'] = wordData.xmax + offset
        this_word_data['start2'] = wordData.xmin
        this_word_data['stop2'] = wordData.xmax
        
        # Add only words, not silences or out-of-vocabulary tokens.
        # Out-of-vocabulary tokens may include names of people, place etc.
        
        if this_word_data['word'] != '':
            this_data[wordCTR + wordStartIdx] = this_word_data
            wordCTR = wordCTR + 1
        
    return len(this_data), this_data

In [ ]:
def getStartTime(wordTimings, tokens):
    
    window = len(tokens)
    
    for wordStartNo in np.arange(len(wordTimings) - window + 1):
        thisWindowWords = []
        thisWindowStarts = []
        for delta in np.arange(window):
            thisWindowWords.append(wordTimings[wordStartNo + delta]['word'])
            thisWindowStarts.append(wordTimings[wordStartNo + delta]['start'])
        if getIntersectionCount(tokens, thisWindowWords)/window > 0.7: 
            if tokens[0] in thisWindowWords:
                startWordIdx = thisWindowWords.index(tokens[0])
                return thisWindowStarts[startWordIdx]
            elif tokens[1] in thisWindowWords:
                startWordIdx = thisWindowWords.index(tokens[1]) - 1
                return thisWindowStarts[startWordIdx]
        
#             return wordTimings[wordStartNo]['start']
    
    return 0

In [ ]:
def getEndTime(wordTimings, tokens):
    
    # Not a great implementations!
    
    window = len(tokens)
    lookAhead = 10
    
    for wordStartNo in np.arange(len(wordTimings) - window + 1):
        thisWindowWords = []
        thisWindowStops = []
        for delta in np.arange(window):
            thisWindowWords.append(wordTimings[wordStartNo + delta]['word'])
            thisWindowStops.append(wordTimings[wordStartNo + delta]['stop'])
        
        if getIntersectionCount(tokens, thisWindowWords)/window > 0.8:
            
#             print("Tokens: ", tokens)
#             print()
#             print("Before look ahead: ", thisWindowWords)
#             print()
            
            # Append some lookahead
            
            for delta in np.arange(lookAhead):
                try:
                    thisWindowWords.append(wordTimings[wordStartNo + delta + window]['word'])
                    thisWindowStops.append(wordTimings[wordStartNo + delta + window]['stop'])
                except:
                    pass
#                     print("\nExceeded bounds.\n")
                    
#             print("After look ahead: ", thisWindowWords)
#             print()
            
            # Now check and return the word end time. Perhaps the following needs to be optimised!
            
            if tokens[-1] in thisWindowWords:
                stopWordIdx = max(loc for loc, val in enumerate(thisWindowWords) if val == tokens[-1])
#                 print("\n Captured: ", thisWindowWords[stopWordIdx])
                return thisWindowStops[stopWordIdx]
            elif tokens[-2] in thisWindowWords:
                stopWordIdx = max(loc for loc, val in enumerate(thisWindowWords) if val == tokens[-2])
#                 print("\n Captured: ", thisWindowWords[stopWordIdx + 1])
                return thisWindowStops[stopWordIdx] # I don't recall, why this isn't thisWindowStops[stopWordIdx + 1]

            
#             return wordTimings[wordStartNo + window - 1]['stop']
        
    return 0